In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%pip show torch

%pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"

Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, easyocr, fastai, kornia, peft, pytorch-ignite, pytorch-lightning, sentence-transformers, stable-baselines3, timm, torchaudio, torchmetrics, torchvision
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-beyc_wlk/unsloth_c9fbba4a7b0c4c72a3d749c4612d1bfd
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-beyc_wlk/unsloth_c9fbba4a7b0c4c72a3d749c4612d1bfd
  Resolved https://github.com/unslothai/unsloth.git to commit e80d642bc777f7a219bdd34aea1a77751f066785
  Installing build dep

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "atharvanighot/Tinyllama-Hindi", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.41G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.70M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

In [4]:
hindi_prompt = """नीचे एक निर्देश दिया गया है जो कार्य का वर्णन करता है। कृपया अनुरोध को उचित रूप से पूरा करने के लिए एक प्रतिक्रिया लिखें।

### निर्देश:
{}


### प्रतिक्रिया:
{}"""

In [12]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    hindi_prompt.format(
        "भारत में लोग कितने त्यौहार मनाते हैं?", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> नीचे एक निर्देश दिया गया है जो कार्य का वर्णन करता है। कृपया अनुरोध को उचित रूप से पूरा करने के लिए एक प्रतिक्रिया लिखें।\n\n### निर्देश:\nन्यूयॉर्क में 3 दिवसीय यात्रा का कार्यक्रम बनाएं।\n\n\n### प्रतिक्रिया:\nयहाँ न्यूयॉर्क में 3 दिनों के लिए मेरा प्रस्तावित कार्यक्रम है:\n\nसप्ताह 1:\n- शुक्रवार सुबह 9:00 बजे मेट्रो स्टेशन पर 1 बजकर 1 मिनट तक पहुँचें।\n- न्यूयॉर्क सिटी में बस जाएँ।\n- मैसाचुकोला में बस लें।\n- बस से लोअर अस्फुलान पार्क में चलें।\n- पार्क में एक बिंगलिंग रेस्तरां में आराम करें।\n- शनिवार रात 8:00 बजे लौटें।\n- शनिवार को छुट्टी।\n- शनिवार की सुबह 9:00 बजे कार में जाएं।\n- कार पर 7:00 बजे अपने गंतव्य पर जाएँ।\n- रविवार को छुट्टी।\n- रविवार की सुबह 7:00 बजे कार में जाएँ।\n- कार से बस जाएँ।\n- दोपहर 10:00 बजे मेट्रो स्टेशन पर पहुंचेंगे।\n- मेट्रो में वापसी ट्रेन लें।\n- सोमवार सुबह 9:00 बजे एक कार में जाएँ।\n- ट्रेन ट्रेन में जाएं।\n- ट्रेन से ट्रेन पर जाएं।\n- ट्रेन पर अपना सामान रखें।\n- रविवार की छुट्टी।\n- रविवार की सुबह 8:00 बजे बस और ट्रेन से बचें।\n- रविवार